In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
def regression_nn_loss(sigma_sq, epsilon = 1e-6):
    def nn_loss(y_true, y_pred):
        return 0.5 * keras.backend.mean(keras.backend.log(sigma_sq + epsilon) + keras.backend.square(y_true - y_pred) / (sigma_sq + epsilon))

    return nn_loss

In [ ]:
# generate dataset
n = 10 # number of features
k = 50 # number of samples
x = 0
np.random.seed(100)
numbers_list = np.array( random.sample(range(100), k) )
while x < n:
    numbers_list = np.vstack((numbers_list, np.array( random.sample(range(100), k) )))
    x = x + 1

dataset = numbers_list.T

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
Y = dataset[:, 0].astype("float32")
X = dataset[:, 1:].astype("float32")

In [ ]:
#np.random.seed(100)
#dataset = np.random.randn(500, 11)
#X = dataset[:,0:10].astype("float32")
#Y = dataset[:,10].astype("float32")

## Network 1

In [ ]:
# solution 1 -- split single neurons

n_tot = int( X.shape[1] )
splits = int( 2 )
n_splits = int( n_tot/splits )

input = keras.layers.Input(shape = (n_tot), name = "input_layer")
layer1 = keras.layers.Lambda(lambda x: x[:,0:n_splits])(input)  # take the first n_splits neurons
layer2 = keras.layers.Lambda(lambda x: x[:,n_splits:])(input)   # take the other neurons
output1 = keras.layers.Dense(units = 1)(layer1)                 # add extra dense layer
output2 = keras.layers.Dense(units = 1)(layer2)                 # add extra dense layer
output = keras.layers.concatenate([output1, output2])           # concatenate the layers to have a single output layer

model = keras.models.Model(input, output)

In [ ]:
print( model.summary() )

In [ ]:
plot_model(model, to_file='model_1.png', show_shapes=True, show_layer_names=True)

In [ ]:
# fit and forecast 
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics=['MeanSquaredError'])
history = model.fit(X, Y, epochs=100)
forecast = model.predict(X)

## Network 2

In [ ]:
# solution 2 -- split entire dataset

input = keras.layers.Input(shape=(n_tot), name = "input_layer")
split = keras.layers.Lambda(lambda x: tf.split(x,num_or_size_splits = splits,axis = 1))(input)
layer1 = keras.layers.Dense(units = 1)(split[0])
layer2 = keras.layers.Dense(units = 1)(split[1])
output = keras.layers.concatenate([layer1, layer2])

model = keras.models.Model(input, output)

In [ ]:
print( model.summary() )

In [ ]:
plot_model(model, to_file='model_2.png', show_shapes=True, show_layer_names=True)

In [ ]:
# fit and forecast 
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics=['MeanSquaredError'])
history = model.fit(X, Y, epochs=100)
forecast = model.predict(X)

## Network 3

In [ ]:
# solution 3 -- separate inputs 
input_1 = keras.layers.Input(shape=(n_splits,))
input_2 = keras.layers.Input(shape=(n_splits,))
layer_1 = keras.layers.Dense(units = 1)(input_1)
layer_2 = keras.layers.Dense(units = 1)(input_2)
output = keras.layers.Add()([layer_1, layer_2])

model = keras.models.Model(inputs=[input_1,input_2], outputs=output)

In [ ]:
print( model.summary() )

In [ ]:
plot_model(model, to_file='model_2.png', show_shapes=True, show_layer_names=True)

In [ ]:
# fit and forecast 
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics=['MeanSquaredError'])
XX = (X[:, :n_splits], X[:, n_splits:])
history = model.fit(XX, Y, epochs=100)
forecast = model.predict(XX)

## Confidence interval for network 1

In [ ]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [ ]:
# confidence interval 
input = keras.layers.Input(shape = (n_tot), name = "input_layer")
layer1 = keras.layers.Lambda(lambda x: x[:,0:n_splits])(input)  # take the first n_splits neurons
layer2 = keras.layers.Lambda(lambda x: x[:,n_splits:])(input)   # take the other neurons
output1 = keras.layers.Dense(units = 1)(layer1)                 # add extra dense layer
output2 = keras.layers.Dense(units = 1)(layer2)                 # add extra dense layer
output = keras.layers.concatenate([output1, output2])           # concatenate the layers to have a single output layer
mean = keras.layers.Dense(units = 1, activation = "linear")(output)
var = keras.layers.Dense(units = 1, activation = "softplus")(output)

train_model = keras.models.Model(input, mean)
pred_model = keras.models.Model(input, [mean, var])

In [ ]:
train_model.compile(loss=regression_nn_loss(var), optimizer="adam")
train_model.fit(X, Y, epochs = 100)
mean, var = pred_model.predict(X)

In [ ]:
li = mean - 1.96*var
ui = mean + 1.96*var
fin = np.concatenate((li, mean, ui), axis = 1)
plt.plot(fin)